# Tutorial16: DIFFPOOL

In [15]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [16]:
import torch

Below are shown the computation to obtain the nodes features matrix and adjacency matrix for the first hierarchical step. 

Initial graph: 
```x_0   = 50 x 32
adj_0  = 50 x 50```

In [17]:
# Node features matrix
x_0 = torch.rand(50, 32)
adj_0 = torch.rand(50,50).round().long()
identity = torch.eye(50)
adj_0 = adj_0 + identity

Set the number of clusters we want to obtain at step 1

50 개의 그래프로부터 5개의 그래프로 줄이겠다.

In [18]:
n_clusters_0 = 50
n_clusters_1 = 5

Initialize the weights of GNN_emb and GNN_pool, we use just 1 conv layer

In [27]:
w_gnn_emb = torch.rand(32, 16)
w_gnn_pool = torch.rand(32, n_clusters_1)

<img src="https://github.com/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial16/img1.png?raw=1" width=300px>




<img src="https://github.com/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial16/img2.png?raw=1" width=400px>


In [32]:
print("adj:\n", adj_0.size(), '\nx_0:\n', x_0.size(), '\nw_gnn_emb:\n', w_gnn_emb.size())

adj:
 torch.Size([50, 50]) 
x_0:
 torch.Size([50, 32]) 
w_gnn_emb:
 torch.Size([32, 16])


In [34]:
print("multiplied:\n", (adj_0@x_0).size())

multiplied:
 torch.Size([50, 32])


In [33]:
z_0 = torch.relu(adj_0 @ x_0 @ w_gnn_emb)
s_0 = torch.softmax(torch.relu(adj_0 @ x_0 @ w_gnn_pool), dim=1)

<img src="https://github.com/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial16/img3.png?raw=1" width=200px>

<img src="https://github.com/AntonioLonga/PytorchGeometricTutorial/blob/main/Tutorial16/img4.png?raw=1" width=200px>

In [21]:
x_1 = s_0.t() @ z_0
adj_1 = s_0.t() @ adj_0 @ s_0

In [22]:
print(x_1.shape)
print(adj_1.shape)

torch.Size([5, 16])
torch.Size([5, 5])


In [23]:
import os.path as osp
from math import ceil

import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
from torch_geometric.nn import DenseGCNConv as GCNConv, dense_diff_pool


In [24]:
max_nodes = 150


class MyFilter(object):
    def __call__(self, data):
        return data.num_nodes <= max_nodes


dataset = TUDataset('data', name='PROTEINS', transform=T.ToDense(max_nodes),
                    pre_filter=MyFilter())
dataset = dataset.shuffle()
n = (len(dataset) + 9) // 10
test_dataset = dataset[:n]
val_dataset = dataset[n:2 * n]
train_dataset = dataset[2 * n:]
test_loader = DenseDataLoader(test_dataset, batch_size=32)
val_loader = DenseDataLoader(val_dataset, batch_size=32)
train_loader = DenseDataLoader(train_dataset, batch_size=32)

In [25]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,
                 normalize=False, lin=True):
        super(GNN, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(GCNConv(in_channels, hidden_channels, normalize))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(GCNConv(hidden_channels, hidden_channels, normalize))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(GCNConv(hidden_channels, out_channels, normalize))
        self.bns.append(torch.nn.BatchNorm1d(out_channels))


    def forward(self, x, adj, mask=None):
        batch_size, num_nodes, in_channels = x.size()
        
        for step in range(len(self.convs)):
            x = self.bns[step](F.relu(self.convs[step](x, adj, mask)))
        

        return x


class DiffPool(torch.nn.Module):
    def __init__(self):
        super(DiffPool, self).__init__()

        num_nodes = ceil(0.25 * max_nodes)
        self.gnn1_pool = GNN(dataset.num_features, 64, num_nodes)
        self.gnn1_embed = GNN(dataset.num_features, 64, 64)

        num_nodes = ceil(0.25 * num_nodes)
        self.gnn2_pool = GNN(64, 64, num_nodes)
        self.gnn2_embed = GNN(64, 64, 64, lin=False)

        self.gnn3_embed = GNN(64, 64, 64, lin=False)

        self.lin1 = torch.nn.Linear(64, 64)
        self.lin2 = torch.nn.Linear(64, dataset.num_classes)

    def forward(self, x, adj, mask=None):
        s = self.gnn1_pool(x, adj, mask)
        x = self.gnn1_embed(x, adj, mask)

        x, adj, l1, e1 = dense_diff_pool(x, adj, s, mask)
        #x_1 = s_0.t() @ z_0
        #adj_1 = s_0.t() @ adj_0 @ s_0
        
        s = self.gnn2_pool(x, adj)
        x = self.gnn2_embed(x, adj)

        x, adj, l2, e2 = dense_diff_pool(x, adj, s)

        x = self.gnn3_embed(x, adj)

        x = x.mean(dim=1)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), l1 + l2, e1 + e2





In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DiffPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output, _, _ = model(data.x, data.adj, data.mask)
        loss = F.nll_loss(output, data.y.view(-1))
        loss.backward()
        loss_all += data.y.size(0) * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0

    for data in loader:
        data = data.to(device)
        pred = model(data.x, data.adj, data.mask)[0].max(dim=1)[1]
        correct += pred.eq(data.y.view(-1)).sum().item()
    return correct / len(loader.dataset)


best_val_acc = test_acc = 0
for epoch in range(1, 151):
    train_loss = train(epoch)
    val_acc = test(val_loader)
    if val_acc > best_val_acc:
        test_acc = test(test_loader)
        best_val_acc = val_acc
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, '
          f'Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

RuntimeError: running_mean should contain 150 elements not 64

발표한 사람이 고치겠다고 해놓고 안고친듯.

그런데 뭐 다른것들이랑 마찬가지로 이것도 우리가 직접 구현해서 돌린다기 보다는 그냥 필요할때 해서 돌리면되니까 괜찮을듯.